In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("cleaned_data.csv")
df.head()

/opt/anaconda3/envs/tf/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,original text,cleaned text,joke
0,0,I am soooo glad I'm not circumcised! My corona...,soooo glad circumcised corona covered foreskin...,1.0
1,1,Did you know Google now has a platform for rec...,know google platform recording bowel movements...,1.0
2,2,What is the difference between my wife and my ...,difference wife vacuum vacuum nt snore sex amp...,1.0
3,3,What did the French man say to the attractive ...,french man say attractive teletubby oo lala,1.0
4,4,"Yo Mama Yo momma's so fat, that when she went ...",yo mama yo momma fat went zoo hippos got jealous,1.0


In [3]:
df = df.drop(columns=df.columns.values[0:2])

In [4]:
df = df.dropna()

In [5]:
df_jokes = df[df['joke'] > 0]
df_non_jokes = df[df['joke'] < 0]

In [6]:
df = pd.concat([df_jokes.sample(25000, random_state=42),
                df_non_jokes.sample(25000, random_state=42)])
df.head()

,cleaned text,joke
544835,walked bar saw fat bird dancing table went sai...,1.0
120452,band antivaxxers refuse listen cure,1.0
100104,call building got engaged premrs,1.0
501226,shit gross one best friend dad used tell us me...,1.0
752947,hear couple stole calendar got six months,1.0


In [7]:
df['joke'].sum()

0.0

In [8]:
X = df.loc[:, 'cleaned text']
y = df.loc[:, 'joke']

# Split Data into Train and Test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42)

In [11]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

40000 40000
10000 10000


In [12]:
X_test[:5]

821108    apple google two brands world valued billion a...
102979    man feel embarrassed twice life first time get...
604087       wish rant sub eligible make front page deleted
425613    people always told make little things count de...
800317    official says us citizen detained north korea ...
Name: cleaned text, dtype: object

In [13]:
y_test[:5]

821108   -1.0
102979    1.0
604087    1.0
425613    1.0
800317   -1.0
Name: joke, dtype: float64

# Tfidf Vectorization

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(ngram_range=(1, 2))

In [17]:
X_train_tfidf = tfidf.fit_transform(X_train)

# Dimensionality Reduction

In [18]:
from sklearn.decomposition import TruncatedSVD

## SVD 200

In [19]:
svd_200 = TruncatedSVD(n_components=200)

In [20]:
X_train_tfidf_svd_200 = svd_200.fit_transform(X_train_tfidf)

In [21]:
# This function preprocesses the test data using the tfidf
# and svd models that were already fit to the training data
def preprocess_test(tfidf, svd, X_test):
    X_test_tfidf = tfidf.transform(X_test)
    X_test_tfidf_svd = svd.transform(X_test_tfidf)
    return X_test_tfidf_svd

In [22]:
X_test_tfidf_svd_200 = preprocess_test(tfidf, svd_200, X_test)

In [23]:
print(X_train_tfidf_svd_200.shape)
print(X_test_tfidf_svd_200.shape)

(40000, 200)
(10000, 200)


In [27]:
np.savetxt('data/svd-200-train.csv', X_train_tfidf_svd_200, delimiter=',')

In [28]:
np.savetxt('data/svd-200-test.csv', X_test_tfidf_svd_200, delimiter=',')

Train and test labels are the same for svd_200 and svd_500 datasets. This only saves it once.

In [31]:
np.savetxt('data/svd-train-labels.csv',y_train,delimiter=',')
np.savetxt('data/svd-test-labels.csv',y_test,delimiter=',')

## SVD 500

In [32]:
svd_500 = TruncatedSVD(n_components=500)

In [33]:
X_train_tfidf_svd_500 = svd_500.fit_transform(X_train_tfidf)

In [34]:
X_test_tfidf_svd_500 = preprocess_test(tfidf, svd_500, X_test)

In [35]:
np.savetxt('data/svd-500-train.csv', X_train_tfidf_svd_500, delimiter=',')
np.savetxt('data/svd-500-test.csv', X_test_tfidf_svd_500, delimiter=',')